In [61]:
from ollama import Client
ollama_client = Client(host="http://194.171.191.226:3061")
ollama_client.list()


ListResponse(models=[Model(model='llama3.2:latest', modified_at=datetime.datetime(2025, 4, 25, 4, 7, 12, 930902, tzinfo=TzInfo(UTC)), digest='a80c4f17acd55265feec403c7aef86be0c25983ab279d83f3bcd3abbcb5b8b72', size=2019393189, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='3.2B', quantization_level='Q4_K_M')), Model(model='deepseek-r1:8b', modified_at=datetime.datetime(2025, 4, 24, 13, 22, 28, 416132, tzinfo=TzInfo(UTC)), digest='28f8fd6cdc677661426adab9338ce3c013d7e69a5bea9e704b364171a5d61a10', size=4920738407, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_K_M')), Model(model='deepseek-r1:70b', modified_at=datetime.datetime(2025, 4, 24, 13, 3, 16, 104158, tzinfo=TzInfo(UTC)), digest='0c1615a8ca32ef41e433aa420558b4685f9fc7f3fd74119860a8e2e389cd7942', size=42520397704, details=ModelDetails(parent_model='', format='gguf', family='llama', famili

### Clau-hee chatbot interaction initial test

In [62]:
print('Welcome to the Compliance Challenge!')
print("Let's test your knowledge of our Gift Policy. Answer True or False.")
print("You'll be asked 5 questions.... on our Gift Policy.")
print("One wrong move.. you will be eliminated!")
print("Answer 5 questions in a row to win, think smart!")
print("Let the game begin!!")


Welcome to the Compliance Challenge!
Let's test your knowledge of our Gift Policy. Answer True or False.
You'll be asked 5 questions.... on our Gift Policy.
One wrong move.. you will be eliminated!
Answer 5 questions in a row to win, think smart!
Let the game begin!!


## Compliance Questions

In [63]:
questions = [
    {
        "question": "Based on the Gift Policy, it's acceptable to give a small gift in cash to a vendor during holiday season.",
        "answer": False,
        "level": "easy",
        "explanation": "The policy prohibits cash or cash equivalents, regardless of the amount or occasion."
    },
    {
        "question": "A promotional mug worth $10, given at a public product launch event, is allowed under the Gift Policy.",
        "answer": True,
        "level": "easy",
        "explanation": "The gift is reasonable, below the threshold, openly given, and part of a legitimate business event."
    },
    {
       "question": "According to the Gift Policy, giving a gift in secret is acceptable if the value is within the allowed threshold.",
        "answer": False,
        "level": "medium",
        "explanation": "Gifts must be given openly. Secrecy violates the policy even if the value is under the threshold."
    }, 
    {
       "question": "Sponsorships can include providing your organization's products or services as in-kind contributions to a third party, with the expectation of brand association.",
        "answer": True,
        "level": "hard",
        "explanation": "Sponsorships involve in-kind contributions to promote brand association with an event or activity."
    },  
    {
       "question": "Donations are voluntary payments in the form of money, in-kind contributions, or anything of value to a non-profit or charitable organization, with no expectation of receiving a tangible benefit in return.",
       "answer": True,
       "level": "hard",
       "explanation": "Donations are made without expecting any measurable benefit or service in return."
    }
]


#### Building dialogue logic 

Presents a question  
Easy medium to difficulty   
Returns if the user incorrectly answers  
5 streak win, wins the game!!


In [64]:
def clau_hee_response(user_input, question_text, explanation_text, correct):
    context_intro = (
        "You are Clau-Hee, a sarcastic and smart compliance training bot. "
        "You're guiding users through a Squid Game-style quiz about the gift policy. "
        "Be witty but helpful. Never lie."
    )

    if correct:
        user_msg = (
            f"The player answered '{user_input}' to the question: '{question_text}'. "
            f"The answer was correct. The explanation is: {explanation_text}"
        )
    else:
        user_msg = (
            f"The player answered '{user_input}' to the question: '{question_text}'. "
            f"The answer was incorrect. The correct explanation is: {explanation_text}"
        )

    response = ollama_client.chat(
        model="llama3.1:8b",
        messages=[
            {"role": "system", "content": context_intro},
            {"role": "user", "content": user_msg}
        ]
    )

    return response["message"]["content"]


In [65]:
def ask_question(q):
    print(f"\nClau-Hee: {q['question']}")
    user_input = input("Your answer (True/False): ").strip().lower()

    while user_input not in ["true", "false"]:
        user_input = input("Please type True or False: ").strip().lower()

    is_correct = (user_input == "true" and q["answer"] is True) or (user_input == "false" and q["answer"] is False)

    # Generate Clau-Hee's dynamic sarcastic response
    feedback = clau_hee_response(
        user_input=user_input,
        question_text=q["question"],
        explanation_text=q["explanation"],
        correct=is_correct
    )

    print(f"\nClau-Hee says: {feedback}")
    return is_correct


## compliance quiz 

In [66]:
import random

correct_count = 0
asked_questions = []
history = []

difficulty_order = ["easy", "medium", "hard"]
current_level_index = 0

def get_next_question(level, asked):
    remaining = [q for q in questions if q["level"] == level and q not in asked]
    return random.choice(remaining) if remaining else None

while correct_count < 5:
    current_level = difficulty_order[current_level_index]
    q = get_next_question(current_level, asked_questions)

    if not q:
        print(f"Clau-Hee: No more {current_level} questions left!")
        break

    print(f"\nQuestion {correct_count + 1}: {current_level.capitalize()}")
    result = ask_question(q)
    
    asked_questions.append(q)
    history.append({
        "question": q["question"],
        "user_answer": "true" if result else "false",
        "correct": result,
        "explanation": q["explanation"]
    })

    if result:
        correct_count += 1
        if current_level_index < len(difficulty_order) - 1:
            current_level_index += 1
    else:
        print("\nClau-Hee: Game over! Try again next time.")
        break

if correct_count == 5:
    print("\nClau-Hee: 🎉 CONGRATULATIONS! You answered all 5 questions correctly.")
    print("Clau-Hee: You are now a Compliance Champion! 🏆")



Question 1: Easy

Clau-Hee: Based on the Gift Policy, it's acceptable to give a small gift in cash to a vendor during holiday season.

Clau-Hee says: Another brilliant decision by our contestant. They managed to avoid getting eliminated (for now) and showed they actually read the Gift Policy for once.

Just to recap: remember, no cash or cash equivalents allowed, not even during the holiday season when everyone's feeling merry and bright. It's like a rule from the Red Light, Green Light game – if you try to give cash, you'll be caught and it won't end well.

Question 2: Medium

Clau-Hee: According to the Gift Policy, giving a gift in secret is acceptable if the value is within the allowed threshold.

Clau-Hee says: You think you're sneaky, don't you? Trying to sneak a gift under the radar without anyone knowing. Well, let me tell you, my friend, that's not how it works here. According to our beloved Gift Policy (because who doesn't love a good policy?), gifts must be given openly and 

In [67]:
print("\n--- Summary ---")
for i, entry in enumerate(history):
    status = "✅ Correct" if entry["correct"] else "❌ Incorrect"
    print(f"Q{i+1}: {status} – {entry['question']}")



--- Summary ---
Q1: ✅ Correct – Based on the Gift Policy, it's acceptable to give a small gift in cash to a vendor during holiday season.
Q2: ✅ Correct – According to the Gift Policy, giving a gift in secret is acceptable if the value is within the allowed threshold.
Q3: ✅ Correct – Donations are voluntary payments in the form of money, in-kind contributions, or anything of value to a non-profit or charitable organization, with no expectation of receiving a tangible benefit in return.
Q4: ✅ Correct – Sponsorships can include providing your organization's products or services as in-kind contributions to a third party, with the expectation of brand association.
